In [2]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

In [3]:
#Dovrebbe funzionare

#Calculates the fft and returns the optimal frequency for the moving average window

def fft_on_diff(profile: xr.core.dataarray.DataArray, 
                average: np.ndarray):
    '''
    Function that takes the difference between the original profile and the smoothed one with moving average as input,
    computes its fft and returns the fft maximum and its correspondent time in order to choose the best window.
    '''
    #Calcoliamo la differenza
    diff_profile = profile-average

    #calcoliamo la trasformata
    fft = np.fft.fft(diff_profile)
    magnitude = np.abs(fft)

    #dare un range per le x
    frequencies = np.linspace(0.4e10, 1.2e10, len(diff_profile))
    time = frequencies
    dt = np.mean(np.diff(time))

    fft_frequencies = np.fft.fftfreq(len(diff_profile), d=dt)

    #plots
    fig, ax = plt.subplots(2, 1, figsize=[10, 5], dpi=200)

    #difference plot
    ax[0].plot(time, diff_profile, color="red", label="Profile minus average")
    ax[0].set_xlabel("Frequency (Hz)") 
    ax[0].set_ylabel("|S21| / dB")
    ax[0].legend()
    ax[0].grid()

    #fft plot
    ax[1].plot(fft_frequencies[fft_frequencies >= 0], 
            magnitude[fft_frequencies >= 0], 
            color='blue', label="FFT")
    ax[1].set_xlabel("Time (s)")
    ax[1].set_ylabel("Magnitude")
    ax[1].legend()
    ax[1].grid()

    fig.tight_layout()
    plt.show()

    #Find maximum
    max_fft = np.max(magnitude[fft_frequencies >= 0])
    max_time = fft_frequencies[fft_frequencies > 0][np.argmax(magnitude[fft_frequencies >= 0])]
    ind_max = np.argmax(magnitude[fft_frequencies >= 0])

    print(f'Massimo\np0: {max_fft}\nTime: {max_time} s\nIndice del massimo: {ind_max}')

    prop = max_fft/max_time

    print(f"Prodotto = {prop}\nFreq = {1/max_time} Hz\n")

    return max_fft, 1/max_time 